In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import pyproj

In [3]:
fp = r'HSL_stops.shp'
bus_stops = gpd.read_file(fp)

fp = r"MetropAccess_YKR_grid_EurefFIN.shp"
grid = gpd.read_file(fp, sep=";")
grid = grid.to_crs(epsg=3857)

#bus_stops = bus_stops.to_crs(epsg=3857)
bus_stops.loc[bus_stops['VERKKO'] == 1]
#bus_stops.loc[(bus_stops["VERKKO"] >= 2) & (bus_stops["VERKKO"] <= 4)]

grid["centroid"] = grid.centroid
grid = grid.set_geometry("centroid")
bus_stops = bus_stops.to_crs(crs=grid.crs)

bus_stops.head(3)

,FID,SOLMUTUNNU,LYHYTTUNNU,SOLMUTYYPP,X,Y,PROJ_X,PROJ_Y,NIMI1,NAMN1,NIMI2,NAMN2,REI_VOIM,AIK_VOIM,VERKKO,PYSAKKITYY,geometry
0,1,1010101,H2997,P,2552977,6673549,2552961,6673548,Yliopisto,Universitetet,Unioninkatu,Unionsgatan,0,0,1,99,POINT (2777545.888 8437715.652)
1,2,1010102,H2007,P,2553276,6673478,2553293,6673478,Ritarihuone,Riddarhuset,Mariankatu,Mariegatan,1,1,1,04,POINT (2778143.320 8437564.392)
2,3,1010103,H2008,P,2553291,6673679,2553279,6673678,Kirkkokatu,Kyrkogatan,Mariankatu 13,Mariegatan 13,1,1,1,04,POINT (2778179.231 8437967.634)


In [12]:
bus_stops.crs == grid.crs
bus_stops.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
grid['centroid'] = grid.centroid
grid.head
#grid.plot()
#bus_stops.plot()
import shapely.speedups
shapely.speedups.enable()

from sklearn.neighbors import BallTree
import numpy as np

def get_nearest(src_points, candidates, k_neighbors=1):

    tree = BallTree(candidates, leaf_size=12, metric='haversine')

    distances, indices = tree.query(src_points, k=k_neighbors)

    # distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # closest indices and distances (i.e. array at index 0)
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)
def nearest_neighbor(frst_gdf, scnd_gdf, return_dist=False):
    """
    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """
    
    frst_geom_col = frst_gdf.geometry.name
    scnd_geom_col = scnd_gdf.geometry.name
    
    scnd = scnd_gdf.copy().reset_index(drop=True)
    
    # Parse coordinates from points and insert them into a numpy array as RADIANS
    frst_radians = np.array(frst_gdf[frst_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    scnd_radians = np.array(scnd[scnd_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    
    # Find the nearest points
    # -----------------------
    # closest ==> index in scnd_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)
    
    closest, dist = get_nearest(src_points=frst_radians, candidates=scnd_radians)

    closest_points = scnd.loc[closest]
    
    
    closest_points = closest_points.reset_index(drop=True)
    
    if return_dist:
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius
        
    return closest_points

In [14]:
import matplotlib.pyplot as plt

In [16]:
closest_stops = nearest_neighbor(grid, bus_stops, return_dist=True)

closest_stops.head(3)
# WHY ARE THE DISTANCES (AND geometry COORDINATES) LIKE THAT??

,FID,SOLMUTUNNU,LYHYTTUNNU,SOLMUTYYPP,X,Y,PROJ_X,PROJ_Y,NIMI1,NAMN1,NIMI2,NAMN2,REI_VOIM,AIK_VOIM,VERKKO,PYSAKKITYY,geometry,distance
0,12437,9651713,Tu6559,P,2558100,6696919,2558094,6696912,Telitie,Telitie,None,None,1,1,1,04,POINT (2788562.805 8484642.683),218272.112305
1,3485,1402131,H3355,P,2557461,6685321,2557464,6685335,"Puistolan as., tulo","Parkstad st., ank.",Tapuliaukio,Stapelplatsen,0,0,1,99,POINT (2786906.148 8461259.568),61603.729949
2,6426,2434301,E4958,P,2537947,6673533,2537955,6673515,Pisanmäki,Bisabacken,Pisantie,Bisavägen,1,1,1,04,POINT (2747408.584 8438056.240),103273.065360
